In [7]:
import cv2
import numpy as np

# Load the binary mask (ensure it's a grayscale image with values 0 or 255)
binary_mask = cv2.imread('mask_907.png', cv2.IMREAD_GRAYSCALE)



In [8]:
num_labels, labels = cv2.connectedComponents(binary_mask)

In [9]:
output_mask = np.zeros((binary_mask.shape[0], binary_mask.shape[1], 3), dtype=np.uint8)

In [10]:
np.random.seed(42)  # For reproducibility
colors = np.random.randint(0, 255, size=(num_labels, 3))  # Create random colors for each label

# Color each instance with a unique color (skip the background label 0)
for label in range(1, num_labels):  # Start from 1 to skip background
    output_mask[labels == label] = colors[label]

# Save or display the colored instance-labeled mask
cv2.imwrite('instance_segmented_colored_mask.png', output_mask)
cv2.imshow('Instance Segmentation with Colors', output_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [14]:
import cv2
import numpy as np
from scipy import ndimage

# Load the binary mask
binary_mask = cv2.imread('mask_883.png', cv2.IMREAD_GRAYSCALE)

# Ensure the mask is binary
# _, binary_mask = cv2.threshold(binary_mask, 127, 255, cv2.THRESH_BINARY)

# Noise removal using morphological operations
kernel = np.ones((3, 3), np.uint8)
sure_bg = cv2.dilate(binary_mask, kernel, iterations=2)

cv2.imshow('Dilation', sure_bg)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [15]:

# Distance transform and thresholding
dist_transform = cv2.distanceTransform(binary_mask, cv2.DIST_L2, 5)
_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# Marker labeling
num_labels, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels to make sure background is not 0, but 1
markers = markers + 1

# Mark the region of unknown with zero
markers[unknown == 255] = 0

# Apply the watershed algorithm
markers = cv2.watershed(cv2.cvtColor(binary_mask, cv2.COLOR_GRAY2BGR), markers)
binary_mask[markers == -1] = [255, 0, 0]  # Boundary marked with red

# Color the output based on labels
output_mask = np.zeros((binary_mask.shape[0], binary_mask.shape[1], 3), dtype=np.uint8)
for label in range(2, num_labels + 1):  # Skip background
    output_mask[markers == label] = np.random.randint(0, 255, size=3)

# Display the output
cv2.imshow('Watershed Segmentation', output_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()


ValueError: NumPy boolean array indexing assignment cannot assign 3 input values to the 2388 output values where the mask is true